In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import time
import pickle
import randomz
import numpy as np
import sys
import glob
import pandas as pd
from operator import itemgetter, attrgetter

## Word2Vec 기반 데이터 가공하여 train / test 데이터 만들기

- pandas로 csv 파일을 읽는다.
- 각 컬럼은 미리 header 와 dtypes를 정의해둔다.
- 그후 base_model_simple 에 맞게 python list 형태를 생성
    - 단점: python list 생성시 메모리가 너무 많이 필요
    - TODO: base_model_simple을 pandas를 직접 읽어서 처리 (python list 버리자)

In [2]:
D = 100

In [3]:
base_header = ["u", "ts", "v", "slot", "label"]
dtypes = {"u":np.int32, "ts":str, "v":np.int32, "slot":np.int32, "label":np.int32}

In [4]:
name_header = base_header
for i in range(D):
    field = "h{}".format(i)
    name_header.append(field)
    dtypes[field] = np.float32
for i in range(D):
    field = "d{}".format(i)
    name_header.append(field)
    dtypes[field] = np.float32

In [5]:
train_df = pd.read_csv(r'd:\WMIND\temp\train_data_w2vec_1M.csv', header=None, names=name_header, dtype=dtypes, parse_dates=['ts'])

In [6]:
col_dic = dict(zip(list(train_df), range(len(list(train_df)))))

In [7]:
train_df = train_df.sample(frac=0.5)

In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 731533 entries, 489494 to 582941
Columns: 205 entries, u to d99
dtypes: datetime64[ns](1), float32(200), int32(4)
memory usage: 580.4 MB


In [9]:
#print(test_df.dtypes)
train_set = []
for row in train_df.itertuples(False):    
    prefix = row[col_dic['u']:col_dic['label']+1]
    profile = row[col_dic['h0']:col_dic['h99']+1]
    deal_feature = row[col_dic['d0']:col_dic['d99']+1]
    train_set.append((prefix, profile, deal_feature))

In [10]:
test_base_header = ["u", "ts", "p_v", "p_slot", "n_v", "n_slot"]
test_dtypes = {"u":np.int32, "ts":str, "p_v":np.int32, "p_slot":np.int32, "n_v":np.int32, "n_slot":np.int32}

In [11]:
name_header = test_base_header
for i in range(D):
    field = "h{}".format(i)
    name_header.append(field)
    test_dtypes[field] = np.float32
for i in range(D):
    field = "p{}".format(i)
    name_header.append(field)
    test_dtypes[field] = np.float32
for i in range(D):
    field = "n{}".format(i)
    name_header.append(field)
    test_dtypes[field] = np.float32

In [12]:
test_df = pd.read_csv(r'd:\WMIND\temp\test_data_w2vec_1M.csv', header=None, names=name_header, dtype=test_dtypes, parse_dates=['ts'])

In [13]:
col_dic = dict(zip(list(test_df), range(len(list(test_df)))))

In [14]:
#print(test_df.dtypes)
test_set = []
for row in test_df.itertuples(False):    
    prefix = row[col_dic['u']:col_dic['n_slot']+1]
    profile = row[col_dic['h0']:col_dic['h99']+1]
    pos_feature = row[col_dic['p0']:col_dic['p99']+1]
    neg_feature = row[col_dic['n0']:col_dic['n99']+1]
    test_set.append((prefix, profile, pos_feature, neg_feature))

### 저장

In [15]:
random.shuffle(train_set)
random.shuffle(test_set)

In [16]:
with open('w2v_dataset.pkl', 'wb') as f:
    pickle.dump(train_set, f, pickle.HIGHEST_PROTOCOL)
    pickle.dump(test_set, f, pickle.HIGHEST_PROTOCOL)